# Training at scale with the Vertex AI Training Service
**Learning Objectives:**
  1. Learn how to organize your training code into a Python package
  1. Train your model using cloud infrastructure via Google Cloud Vertex AI Training Service
  1. (optional) Learn how to run your training package using Docker containers and push training Docker images on a Docker registry

## Introduction

In this notebook we'll make the jump from training locally, to do training in the cloud. We'll take advantage of Google Cloud's [Vertex AI Training Service](https://cloud.google.com/vertex-ai/). 

Vertex AI Training Service is a managed service that allows the training and deployment of ML models without having to provision or maintain servers. The infrastructure is handled seamlessly by the managed service for us.

Each learning objective will correspond to a __#TODO__ in the [student lab notebook](../labs/1_training_at_scale_vertex.ipynb) -- try to complete that notebook first before reviewing this solution notebook.

Specify your project name and bucket name in the cell below.

In [15]:
import os
import warnings

from google import api_core
from google.cloud import bigquery

warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

Change the following cell as necessary:

In [16]:
# Change below if necessary
PROJECT = !gcloud config get-value project  # noqa: E999
PROJECT = PROJECT[0]
BUCKET = PROJECT
REGION = "us-central1"

OUTDIR = f"gs://{BUCKET}/taxifare/data"

%env PROJECT=$PROJECT
%env BUCKET=$BUCKET
%env REGION=$REGION
%env OUTDIR=$OUTDIR
%env TFVERSION=2.8

env: PROJECT=qwiklabs-asl-01-19968276eb55
env: BUCKET=qwiklabs-asl-01-19968276eb55
env: REGION=us-central1
env: OUTDIR=gs://qwiklabs-asl-01-19968276eb55/taxifare/data
env: TFVERSION=2.8


Confirm below that the bucket is regional and its region equals to the specified region:

In [17]:
%%bash
gsutil ls -Lb gs://$BUCKET | grep "gs://\|Location"
echo $REGION

gs://qwiklabs-asl-01-19968276eb55/ :
	Location type:			region
	Location constraint:		US-CENTRAL1
us-central1


In [18]:
%%bash
gcloud config set project $PROJECT
gcloud config set ai/region $REGION

Updated property [core/project].
Updated property [ai/region].


## Create BigQuery tables

If you have not already created a BigQuery dataset for our data, run the following cell:

In [19]:
bq = bigquery.Client(project=PROJECT)
dataset = bigquery.Dataset(bq.dataset("taxifare"))

try:
    bq.create_dataset(dataset)
    print("Dataset created")
except api_core.exceptions.Conflict:
    print("Dataset already exists")

Dataset already exists


Let's create a table with 1 million examples.

Note that the order of columns is exactly what was in our CSV files.

In [26]:
%%bigquery

CREATE OR REPLACE TABLE taxifare.feateng_training_data AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    'unused' AS key
FROM `nyc-tlc.yellow.trips` 
WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 1000)) = 1
AND
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0

Query is running:   0%|          |

""


Make the validation dataset be 1/10 the size of the training dataset.

In [27]:
%%bigquery

CREATE OR REPLACE TABLE taxifare.feateng_valid_data AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    'unused' AS key
FROM `nyc-tlc.yellow.trips`
WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 10000)) = 2
AND
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0

Query is running:   0%|          |

""


## Export the tables as CSV files

In [28]:
%%bash

echo "Deleting current contents of $OUTDIR"
gsutil -m -q rm -rf $OUTDIR

echo "Extracting training data to $OUTDIR"
bq --location=US extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   taxifare.feateng_training_data \
   $OUTDIR/taxi-train-*.csv

echo "Extracting validation data to $OUTDIR"
bq --location=US extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   taxifare.feateng_valid_data \
   $OUTDIR/taxi-valid-*.csv

gsutil ls -l $OUTDIR

Deleting current contents of gs://qwiklabs-asl-01-19968276eb55/taxifare/data
Extracting training data to gs://qwiklabs-asl-01-19968276eb55/taxifare/data


Waiting on bqjob_r74c2fa7e06830783_00000196cc69b39c_1 ... (5s) Current status: DONE   


Extracting validation data to gs://qwiklabs-asl-01-19968276eb55/taxifare/data


Waiting on bqjob_r1c7980fe77045427_00000196cc69e1c7_1 ... (1s) Current status: DONE   


  29483349  2025-05-14T01:30:05Z  gs://qwiklabs-asl-01-19968276eb55/taxifare/data/taxi-train-000000000000.csv
  29387484  2025-05-14T01:30:07Z  gs://qwiklabs-asl-01-19968276eb55/taxifare/data/taxi-train-000000000001.csv
  29474402  2025-05-14T01:30:07Z  gs://qwiklabs-asl-01-19968276eb55/taxifare/data/taxi-train-000000000002.csv
   8725746  2025-05-14T01:30:15Z  gs://qwiklabs-asl-01-19968276eb55/taxifare/data/taxi-valid-000000000000.csv
TOTAL: 4 objects, 97070981 bytes (92.57 MiB)


Confirm that you have created both the training and validation datasets in Google Cloud Storage.

In [29]:
!gsutil ls gs://$BUCKET/taxifare/data

gs://qwiklabs-asl-01-19968276eb55/taxifare/data/taxi-train-000000000000.csv
gs://qwiklabs-asl-01-19968276eb55/taxifare/data/taxi-train-000000000001.csv
gs://qwiklabs-asl-01-19968276eb55/taxifare/data/taxi-train-000000000002.csv
gs://qwiklabs-asl-01-19968276eb55/taxifare/data/taxi-valid-000000000000.csv


In [30]:
!gsutil cat gs://$BUCKET/taxifare/data/taxi-train-000000000000.csv | head -2

2.5,2012-01-12 17:53:21 UTC,-74.005409,40.748532,-74.004533,40.748225,1,unused
2.5,2012-08-14 20:00:38 UTC,-73.974073,40.679595,-73.974937,40.680728,1,unused


In [31]:
!gsutil cat gs://$BUCKET/taxifare/data/taxi-valid-000000000000.csv | head -2

2.5,2012-05-30 19:22:00 UTC,-73.98416,40.764733,-73.982348,40.76473,1,unused
2.5,2013-10-27 03:25:00 UTC,-74.003092,40.727735,-74.00236,40.72805,1,unused


## Make code compatible with Vertex AI Training Service
In order to make our code compatible with Vertex AI Training Service we need to make the following changes:

1. Upload data to Google Cloud Storage 
2. Move code into a trainer Python package
4. Submit training job with `gcloud` to train on Vertex AI

### Move code into a python package

The first thing to do is to convert your training code snippets into a regular Python package. 

A Python package is simply a collection of one or more `.py` files along with an `__init__.py` file to identify the containing directory as a package. The `__init__.py` sometimes contains initialization code but for our purposes an empty file suffices.

#### Create the package directory

Our package directory contains 3 files:

In [ ]:
ls ./taxifare/trainer/

#### Paste existing code into model.py

A Python package requires our code to be in a .py file, as opposed to notebook cells. So, we simply copy and paste our existing code for the previous notebook into a single file.

In the cell below, we write the contents of the cell into `model.py` packaging the model we 
developed in the previous labs so that we can deploy it to Vertex AI Training Service.  

**Lab Task #1**: Organizing your training code into a Python package

There are two places to fill in TODOs in `model.py`. 

 * in the `build_dnn_model` function, add code to use a optimizer with a custom learning rate.
 * in the `train_and_evaluate` function, add code to define variables using the `hparams` dictionary.

In [38]:
%%writefile ./taxifare/trainer/model.py
"""Data prep, train and evaluate DNN model."""

import logging
import os

import numpy as np
import tensorflow as tf
from tensorflow.keras import callbacks,  models
from tensorflow.keras.layers import (
    Concatenate,
    Dense,
    Discretization,
    Embedding,
    Flatten,
    Input,
    Lambda,
)
from tensorflow.keras.layers.experimental.preprocessing import HashedCrossing

logging.info(tf.version.VERSION)

CSV_COLUMNS = [
    "fare_amount",
    "pickup_datetime",
    "pickup_longitude",
    "pickup_latitude",
    "dropoff_longitude",
    "dropoff_latitude",
    "passenger_count",
    "key",
]

LABEL_COLUMN = "fare_amount"
DEFAULTS = [[0.0], ["na"], [0.0], [0.0], [0.0], [0.0], [0.0], ["na"]]
UNWANTED_COLS = ["pickup_datetime", "key"]

INPUT_COLS = [
    c for c in CSV_COLUMNS if c != LABEL_COLUMN and c not in UNWANTED_COLS
]

def features_and_labels(row_data):
    for unwanted_col in UNWANTED_COLS:
        row_data.pop(unwanted_col)
    label = row_data.pop(LABEL_COLUMN)
    return row_data, label


def load_dataset(pattern, batch_size, num_repeat):
    dataset = tf.data.experimental.make_csv_dataset(
        file_pattern=pattern,
        batch_size=batch_size,
        column_names=CSV_COLUMNS,
        column_defaults=DEFAULTS,
        num_epochs=num_repeat,
        shuffle_buffer_size=1000000,
    )
    return dataset.map(features_and_labels)


def create_train_dataset(pattern, batch_size):
    dataset = load_dataset(pattern, batch_size, num_repeat=None)
    return dataset.prefetch(1)


def create_eval_dataset(pattern, batch_size):
    dataset = load_dataset(pattern, batch_size, num_repeat=1)
    return dataset.prefetch(1)


def euclidean(params):
    lon1, lat1, lon2, lat2 = params
    londiff = lon2 - lon1
    latdiff = lat2 - lat1
    return tf.sqrt(londiff * londiff + latdiff * latdiff)


def scale_longitude(lon_column):
    return (lon_column + 78) / 8.0


def scale_latitude(lat_column):
    return (lat_column - 37) / 8.0


def transform(inputs, nbuckets):
    transformed = {}

    # Scaling longitude from range [-70, -78] to [0, 1]
    transformed["scaled_plon"] = Lambda(scale_longitude, name="scale_plon")(
        inputs["pickup_longitude"]
    )
    transformed["scaled_dlon"] = Lambda(scale_longitude, name="scale_dlon")(
        inputs["dropoff_longitude"]
    )

    # Scaling latitude from range [37, 45] to [0, 1]
    transformed["scaled_plat"] = Lambda(scale_latitude, name="scale_plat")(
        inputs["pickup_latitude"]
    )
    transformed["scaled_dlat"] = Lambda(scale_latitude, name="scale_dlat")(
        inputs["dropoff_latitude"]
    )

    # Apply euclidean function
    transformed["euclidean_distance"] = Lambda(euclidean, name="euclidean")(
        [
            inputs["pickup_longitude"],
            inputs["pickup_latitude"],
            inputs["dropoff_longitude"],
            inputs["dropoff_latitude"],
        ]
    )

    latbuckets = np.linspace(start=0.0, stop=1.0, num=nbuckets).tolist()
    lonbuckets = np.linspace(start=0.0, stop=1.0, num=nbuckets).tolist()

    # Bucketization with Discretization layer
    plon = Discretization(lonbuckets, name="plon_bkt")(
        transformed["scaled_plon"]
    )
    plat = Discretization(latbuckets, name="plat_bkt")(
        transformed["scaled_plat"]
    )
    dlon = Discretization(lonbuckets, name="dlon_bkt")(
        transformed["scaled_dlon"]
    )
    dlat = Discretization(latbuckets, name="dlat_bkt")(
        transformed["scaled_dlat"]
    )

    # Feature Cross with HashedCrossing layer
    p_fc = HashedCrossing(num_bins=nbuckets * nbuckets, name="p_fc")(
        (plon, plat)
    )
    d_fc = HashedCrossing(num_bins=nbuckets * nbuckets, name="d_fc")(
        (dlon, dlat)
    )
    pd_fc = HashedCrossing(num_bins=nbuckets**4, name="pd_fc")((p_fc, d_fc))

    # Embedding with Embedding layer
    transformed["pd_embed"] = Flatten()(
        Embedding(input_dim=nbuckets**4, output_dim=10, name="pd_embed")(
            pd_fc
        )
    )

    transformed["passenger_count"] = inputs["passenger_count"]
    
    return transformed


def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))


def build_dnn_model(nbuckets, nnsize, lr):
    inputs = {
        colname: Input(name=colname, shape=(1,), dtype="float32")
        for colname in INPUT_COLS
    }

    # transforms
    transformed = transform(inputs, nbuckets)
    dnn_inputs = Concatenate()(transformed.values())

    x = dnn_inputs
    for layer, nodes in enumerate(nnsize):
        x = Dense(nodes, activation="relu", name=f"h{layer}")(x)
    output = Dense(1, name="fare")(x)

    model = models.Model(inputs, output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="mse", )

    return model


def train_and_evaluate(hparams):
    # TODO 1b: Your code here
    nbuckets = hparams["nbuckets"]
    nnsize = hparams["nnsize"]
    lr = hparams["lr"]
    batch_size = hparams["batch_size"]

    nnsize = [int(s) for s in hparams["nnsize"].split()]
    eval_data_path = hparams["eval_data_path"]
    num_evals = hparams["num_evals"]
    num_examples_to_train_on = hparams["num_examples_to_train_on"]
    output_dir = hparams["output_dir"]
    train_data_path = hparams["train_data_path"]

    model_export_path = os.path.join(output_dir, "savedmodel")
    checkpoint_path = os.path.join(output_dir, "checkpoints")
    tensorboard_path = os.path.join(output_dir, "tensorboard")

    if tf.io.gfile.exists(output_dir):
        tf.io.gfile.rmtree(output_dir)

    model = build_dnn_model(nbuckets, nnsize, lr)
    logging.info(model.summary())

    trainds = create_train_dataset(train_data_path, batch_size)
    evalds = create_eval_dataset(eval_data_path, batch_size)

    steps_per_epoch = num_examples_to_train_on // (batch_size * num_evals)

    checkpoint_cb = callbacks.ModelCheckpoint(
        checkpoint_path, save_weights_only=True, verbose=1
    )
    tensorboard_cb = callbacks.TensorBoard(tensorboard_path, histogram_freq=1)

    history = model.fit(
        trainds,
        validation_data=evalds,
        epochs=num_evals,
        steps_per_epoch=max(1, steps_per_epoch),
        verbose=2,  # 0=silent, 1=progress bar, 2=one line per epoch
        callbacks=[checkpoint_cb, tensorboard_cb],
    )

    # Exporting the model with default serving function.
    model.save(model_export_path)
    return history


Overwriting ./taxifare/trainer/model.py


### Modify code to read data from and write checkpoint files to GCS 

If you look closely above, you'll notice a new function, `train_and_evaluate` that wraps the code that actually trains the model. This allows us to parametrize the training by passing a dictionary of parameters to this function (e.g, `batch_size`, `num_examples_to_train_on`, `train_data_path` etc.)

This is useful because the output directory, data paths and number of train steps will be different depending on whether we're training locally or in the cloud. Parametrizing allows us to use the same code for both.

We specify these parameters at run time via the command line. Which means we need to add code to parse command line parameters and invoke `train_and_evaluate()` with those params. This is the job of the `task.py` file. 

In [39]:
%%writefile taxifare/trainer/task.py
"""Argument definitions for model training code in `trainer.model`."""

import argparse

from trainer import model

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--batch_size",
        help="Batch size for training steps",
        type=int,
        default=32,
    )
    parser.add_argument(
        "--eval_data_path",
        help="GCS location pattern of eval files",
        required=True,
    )
    parser.add_argument(
        "--nnsize",
        help="Hidden layer sizes (provide space-separated sizes)",
        default="32 8",
    )
    parser.add_argument(
        "--nbuckets",
        help="Number of buckets to divide lat and lon with",
        type=int,
        default=10,
    )
    parser.add_argument(
        "--lr", help="learning rate for optimizer", type=float, default=0.001
    )
    parser.add_argument(
        "--num_evals",
        help="Number of times to evaluate model on eval data training.",
        type=int,
        default=5,
    )
    parser.add_argument(
        "--num_examples_to_train_on",
        help="Number of examples to train on.",
        type=int,
        default=100,
    )
    parser.add_argument(
        "--output_dir",
        help="GCS location to write checkpoints and export models",
        required=True,
    )
    parser.add_argument(
        "--train_data_path",
        help="GCS location pattern of train files containing eval URLs",
        required=True,
    )
    args = parser.parse_args()
    hparams = args.__dict__

    model.train_and_evaluate(hparams)


Overwriting taxifare/trainer/task.py


### Run trainer module package locally

Now we can test our training code locally as follows using the local test data. We'll run a very small training job over a single file with a small batch size and one eval step.

In [40]:
%%bash

EVAL_DATA_PATH=../data/taxi-traffic-valid*
TRAIN_DATA_PATH=../data/taxi-traffic-train*
OUTPUT_DIR=./taxifare-model

test ${OUTPUT_DIR} && rm -rf ${OUTPUT_DIR}
export PYTHONPATH=${PYTHONPATH}:${PWD}/taxifare
    
python3 -m trainer.task \
--eval_data_path $EVAL_DATA_PATH \
--output_dir $OUTPUT_DIR \
--train_data_path $TRAIN_DATA_PATH \
--batch_size 5 \
--num_examples_to_train_on 10000 \
--num_evals 1 \
--nbuckets 10 \
--lr 0.001 \
--nnsize "32 8"

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 pickup_longitude (InputLayer)  [(None, 1)]          0           []                               
                                                                                                  
 dropoff_longitude (InputLayer)  [(None, 1)]         0           []                               
                                                                                                  
 pickup_latitude (InputLayer)   [(None, 1)]          0           []                               
                                                                                                  
 dropoff_latitude (InputLayer)  [(None, 1)]          0           []                               
                                                                                              

### Run your training package on Vertex AI using a pre-built container

Once the code works in standalone mode locally, you can run it on the Cloud using Vertex AI and use pre-built containers. First, we need to package our code as a source distribution. For this, we can use `setuptools`. 

In [41]:
%%writefile taxifare/setup.py
"""Using `setuptools` to create a source distribution."""

from setuptools import find_packages, setup

setup(
    name="taxifare_trainer",
    version="0.1",
    packages=find_packages(),
    include_package_data=True,
    description="Taxifare model training application.",
)

Writing taxifare/setup.py


In [42]:
%%bash
cd taxifare
python ./setup.py sdist --formats=gztar
cd ..

running sdist
running egg_info
creating taxifare_trainer.egg-info
writing taxifare_trainer.egg-info/PKG-INFO
writing dependency_links to taxifare_trainer.egg-info/dependency_links.txt
writing top-level names to taxifare_trainer.egg-info/top_level.txt
writing manifest file 'taxifare_trainer.egg-info/SOURCES.txt'
reading manifest file 'taxifare_trainer.egg-info/SOURCES.txt'
writing manifest file 'taxifare_trainer.egg-info/SOURCES.txt'


running check
creating taxifare_trainer-0.1
creating taxifare_trainer-0.1/taxifare_trainer.egg-info
creating taxifare_trainer-0.1/trainer
copying files to taxifare_trainer-0.1...
copying setup.py -> taxifare_trainer-0.1
copying taxifare_trainer.egg-info/PKG-INFO -> taxifare_trainer-0.1/taxifare_trainer.egg-info
copying taxifare_trainer.egg-info/SOURCES.txt -> taxifare_trainer-0.1/taxifare_trainer.egg-info
copying taxifare_trainer.egg-info/dependency_links.txt -> taxifare_trainer-0.1/taxifare_trainer.egg-info
copying taxifare_trainer.egg-info/top_level.txt -> taxifare_trainer-0.1/taxifare_trainer.egg-info
copying trainer/__init__.py -> taxifare_trainer-0.1/trainer
copying trainer/model.py -> taxifare_trainer-0.1/trainer
copying trainer/task.py -> taxifare_trainer-0.1/trainer
copying taxifare_trainer.egg-info/SOURCES.txt -> taxifare_trainer-0.1/taxifare_trainer.egg-info
Writing taxifare_trainer-0.1/setup.cfg
creating dist
Creating tar archive
removing 'taxifare_trainer-0.1' (and everythi

We will store our package in the Cloud Storage bucket.

In [43]:
%%bash
gsutil cp taxifare/dist/taxifare_trainer-0.1.tar.gz gs://${BUCKET}/taxifare/

Copying file://taxifare/dist/taxifare_trainer-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  3.4 KiB/  3.4 KiB]                                                
Operation completed over 1 objects/3.4 KiB.                                      


#### Submit Custom Job using the `gcloud` CLI

To submit this source distribution the Cloud we use [`gcloud ai custom-jobs create`](https://cloud.google.com/sdk/gcloud/reference/ai/custom-jobs/create) and simply specify some additional parameters for Vertex AI Training service:
- job_name: A unique identifier for the Cloud job. We usually append system time to ensure uniqueness
- region: Cloud region to train in. See [here](https://cloud.google.com/vertex-ai/docs/general/locations) for supported Vertex AI Custom model training regions

The arguments within `--args` are sent to our `task.py`.

Because this is on the entire dataset, it will take a while. You can monitor the job from the GCP console in the Vertex AI Training section.

**Lab Task #2**: Train your model using cloud infrastructure via Google Cloud Vertex AI Training Service
Fill in the TODOs in the code below to submit your job for training on Vertex AI. 

In [45]:
%%bash

# Output directory and jobID
TIMESTAMP=$(date -u +%Y%m%d_%H%M%S)
OUTDIR=gs://${BUCKET}/taxifare/trained_model_$TIMESTAMP
JOB_NAME=taxifare_$TIMESTAMP
echo ${OUTDIR} ${REGION} ${JOB_NAME}

PYTHON_PACKAGE_URIS=gs://${BUCKET}/taxifare/taxifare_trainer-0.1.tar.gz
MACHINE_TYPE=n1-standard-4
REPLICA_COUNT=1
PYTHON_PACKAGE_EXECUTOR_IMAGE_URI="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-12.py310:latest"
PYTHON_MODULE=trainer.task

# Model and training hyperparameters
BATCH_SIZE=50
NUM_EXAMPLES_TO_TRAIN_ON=5000
NUM_EVALS=100
NBUCKETS=10
LR=0.001
NNSIZE="32 8"

# GCS paths
GCS_PROJECT_PATH=gs://$BUCKET/taxifare
DATA_PATH=$GCS_PROJECT_PATH/data
TRAIN_DATA_PATH=$DATA_PATH/taxi-train*
EVAL_DATA_PATH=$DATA_PATH/taxi-valid*

WORKER_POOL_SPEC="machine-type=$MACHINE_TYPE,\
replica-count=$REPLICA_COUNT,\
executor-image-uri=$PYTHON_PACKAGE_EXECUTOR_IMAGE_URI,\
python-module=$PYTHON_MODULE"

ARGS="--eval_data_path=$EVAL_DATA_PATH,\
--output_dir=$OUTDIR,\
--train_data_path=$TRAIN_DATA_PATH,\
--batch_size=$BATCH_SIZE,\
--num_examples_to_train_on=$NUM_EXAMPLES_TO_TRAIN_ON,\
--num_evals=$NUM_EVALS,\
--nbuckets=$NBUCKETS,\
--lr=$LR,\
--nnsize=$NNSIZE"

gcloud ai custom-jobs create \
  --region=${REGION} \
  --display-name=$JOB_NAME \
  --python-package-uris=$PYTHON_PACKAGE_URIS \
  --worker-pool-spec=$WORKER_POOL_SPEC \
  --args="$ARGS"

gs://qwiklabs-asl-01-19968276eb55/taxifare/trained_model_20250514_025029 us-central1 taxifare_20250514_025029


Using endpoint [https://us-central1-aiplatform.googleapis.com/]
CustomJob [projects/604342147284/locations/us-central1/customJobs/6477529367835574272] is submitted successfully.

Your job is still active. You may view the status of your job with the command

  $ gcloud ai custom-jobs describe projects/604342147284/locations/us-central1/customJobs/6477529367835574272

or continue streaming the logs with the command

  $ gcloud ai custom-jobs stream-logs projects/604342147284/locations/us-central1/customJobs/6477529367835574272


### Run your training package using a custom container

Vertex AI Training also supports training in custom containers, allowing users to bring their own Docker containers with any pre-installed ML framework or algorithm to run on Vertex AI Training. 

In this last section, we'll see how to submit a Cloud training job using a customized Docker image. 

Containerizing our `./taxifare/trainer` package involves 3 steps:

* Writing a Dockerfile in `./taxifare`
* Building the Docker image
* Pushing it to the Google Cloud Artifact Registry in our GCP project

The `Dockerfile` specifies
1. How the container needs to be provisioned so that all the dependencies in our code are satisfied
2. Where to copy our trainer Package in the container
3. What command to run when the container is ran (the `ENTRYPOINT` line)

**Lab Task #3**: Running your training package using Docker containers.
Fill in the TODOs in the code below for Dockerfile

In [46]:
%%writefile ./taxifare/Dockerfile
FROM us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-12.py310:latest

COPY . /code

WORKDIR /code

ENTRYPOINT ["python3", "-m", "trainer.task"]

Writing ./taxifare/Dockerfile


In [47]:
%%bash 

PROJECT_DIR=$(cd ./taxifare && pwd)
ARTIFACT_REGISTRY_DIR=asl-artifact-repo
IMAGE_NAME=taxifare_training_container
DOCKERFILE=$PROJECT_DIR/Dockerfile
IMAGE_URI=us-docker.pkg.dev/$PROJECT/$ARTIFACT_REGISTRY_DIR/$IMAGE_NAME:latest

# Authorize Artifact Registry
gcloud auth print-access-token | docker login -u oauth2accesstoken --password-stdin https://us-docker.pkg.dev

docker build $PROJECT_DIR -f $DOCKERFILE -t $IMAGE_URI

docker push $IMAGE_URI

WARNING! Your password will be stored unencrypted in /home/jupyter/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credential-stores



Login Succeeded


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 181B done
#1 DONE 0.0s

#2 [internal] load metadata for us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-12.py310:latest
#2 DONE 0.2s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [internal] load build context
#4 transferring context: 106.31kB done
#4 DONE 0.0s

#5 [1/3] FROM us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-12.py310:latest@sha256:edfbaf60dff875781ddd03a7e5683e03feaea0e99a3452db6cf9ba3077fa0e3f
#5 resolve us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-12.py310:latest@sha256:edfbaf60dff875781ddd03a7e5683e03feaea0e99a3452db6cf9ba3077fa0e3f 0.0s done
#5 sha256:edfbaf60dff875781ddd03a7e5683e03feaea0e99a3452db6cf9ba3077fa0e3f 5.96kB / 5.96kB done
#5 sha256:7a4eda0e06178110d3c1ab6255fb12d88111745d0a643eef1ef4a9732a72e272 14.18kB / 14.18kB done
#5 sha256:7478e0ac0f23f94b2f27848fbcdf804a670fbf8d4bab26df842d40a10c

The push refers to repository [us-docker.pkg.dev/qwiklabs-asl-01-19968276eb55/asl-artifact-repo/taxifare_training_container]
5f70bf18a086: Preparing
7fb0116cb477: Preparing
e42695c7b436: Preparing
e42695c7b436: Preparing
7e34967c8575: Preparing
4e316f7a8cfd: Preparing
aabd5006c821: Preparing
4d19c75cc81d: Preparing
4d19c75cc81d: Preparing
43787889dfe1: Preparing
e51967d24b89: Preparing
34555790f018: Preparing
8536d85d1fc7: Preparing
5d5447dbfb9f: Preparing
17cade2859ad: Preparing
e432beef45ab: Preparing
f4dec8971824: Preparing
f4dec8971824: Preparing
4b4661d32244: Preparing
5f70bf18a086: Preparing
4d8c0fa3291e: Preparing
6647ad4a8716: Preparing
6bb43c4536cc: Preparing
49fd3d481bad: Preparing
2158942577bb: Preparing
6fadc2de3475: Preparing
c2000232ca5b: Preparing
d84a35e62906: Preparing
2573e0d81582: Preparing
aabd5006c821: Waiting
4d19c75cc81d: Waiting
43787889dfe1: Waiting
e51967d24b89: Waiting
34555790f018: Waiting
8536d85d1fc7: Waiting
5d5447dbfb9f: Waiting
4b4661d32244: Waiting
17c

**Remark:** If you prefer to build the container image from the command line, we have written a script for that `./taxifare/scripts/build.sh`. This script reads its configuration from the file `./taxifare/scripts/env.sh`. You can configure these arguments the way you want in that file. You can also simply type `make build` from within `./taxifare` to build the image (which will invoke the build script). Similarly, we wrote the script `./taxifare/scripts/push.sh` to push the Docker image, which you can also trigger by typing `make push` from within `./taxifare`.

#### Train using a custom container on Vertex AI

TODO: To submit to the Cloud we use [`gcloud ai custom-jobs create`](https://cloud.google.com/sdk/gcloud/reference/ai/custom-jobs/create) and simply specify some additional parameters for Vertex AI Training Service:
- job_name: A unique identifier for the Cloud job. We usually append system time to ensure uniqueness
- image-uri: The uri of the Docker image we pushed in the Google Cloud registry
- region: Cloud region to train in. See [here](https://cloud.google.com/vertex-ai/docs/general/locations) for supported Vertex AI Training Service regions

The arguments within `--args` are sent to our `task.py`.

You can track your job and view logs using [cloud console](https://console.cloud.google.com/mlengine/jobs).

In [48]:
%%bash

# Output directory and jobID
TIMESTAMP=$(date -u +%Y%m%d_%H%M%S)
OUTDIR=gs://${BUCKET}/taxifare/trained_model_$TIMESTAMP
JOB_NAME=taxifare_container_$TIMESTAMP
echo ${OUTDIR} ${REGION} ${JOB_NAME}

# Model and training hyperparameters
BATCH_SIZE=50
NUM_EXAMPLES_TO_TRAIN_ON=5000
NUM_EVALS=100
NBUCKETS=10
LR=0.001
NNSIZE="32 8"

# Vertex AI machines to use for training
MACHINE_TYPE=n1-standard-4
REPLICA_COUNT=1

# GCS paths.
GCS_PROJECT_PATH=gs://$BUCKET/taxifare
DATA_PATH=$GCS_PROJECT_PATH/data
TRAIN_DATA_PATH=$DATA_PATH/taxi-train*
EVAL_DATA_PATH=$DATA_PATH/taxi-valid*

ARTIFACT_REGISTRY_DIR=asl-artifact-repo
IMAGE_NAME=taxifare_training_container
IMAGE_URI=us-docker.pkg.dev/$PROJECT/$ARTIFACT_REGISTRY_DIR/$IMAGE_NAME:latest

WORKER_POOL_SPEC="machine-type=$MACHINE_TYPE,\
replica-count=$REPLICA_COUNT,\
container-image-uri=$IMAGE_URI"

ARGS="--eval_data_path=$EVAL_DATA_PATH,\
--output_dir=$OUTDIR,\
--train_data_path=$TRAIN_DATA_PATH,\
--batch_size=$BATCH_SIZE,\
--num_examples_to_train_on=$NUM_EXAMPLES_TO_TRAIN_ON,\
--num_evals=$NUM_EVALS,\
--nbuckets=$NBUCKETS,\
--lr=$LR,\
--nnsize=$NNSIZE"

gcloud ai custom-jobs create \
  --region=$REGION \
  --display-name=$JOB_NAME \
  --worker-pool-spec=$WORKER_POOL_SPEC \
  --args="$ARGS"

gs://qwiklabs-asl-01-19968276eb55/taxifare/trained_model_20250514_025453 us-central1 taxifare_container_20250514_025453


Using endpoint [https://us-central1-aiplatform.googleapis.com/]
CustomJob [projects/604342147284/locations/us-central1/customJobs/949360825238290432] is submitted successfully.

Your job is still active. You may view the status of your job with the command

  $ gcloud ai custom-jobs describe projects/604342147284/locations/us-central1/customJobs/949360825238290432

or continue streaming the logs with the command

  $ gcloud ai custom-jobs stream-logs projects/604342147284/locations/us-central1/customJobs/949360825238290432


Copyright 2023 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License